# CISC 351/372 Advanced Data Analytics Group Project

## Group 8: Political Sentiment Analysis In Liberal and Conservative Reddit Communities

### RQ3: How does the use of hyperbolic language differ between liberal and conservative Reddit communities?

This notebook contains the code to answer the third research question.

#### Imports

In [1]:
%pip install nrclex
%pip install nltk NRCLex tqdm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import pandas as pd
import re
import matplotlib.pyplot as plt
import nltk
import random


from tqdm import tqdm
from joblib import Parallel, delayed
from collections import Counter
from nltk import word_tokenize, pos_tag

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')




A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\riple\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\riple\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "c:\Users\riple\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start
    self.io_loop.start()
  File "c:\Users\riple\anaconda3\Lib\site-packa

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\riple\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\riple\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "c:\Users\riple\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start
    self.io_loop.start()
  File "c:\Users\riple\anaconda3\Lib\site-packa

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\riple\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\riple\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "c:\Users\riple\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start
    self.io_loop.start()
  File "c:\Users\riple\anaconda3\Lib\site-packa

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\riple\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\riple\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "c:\Users\riple\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start
    self.io_loop.start()
  File "c:\Users\riple\anaconda3\Lib\site-packa

AttributeError: _ARRAY_API not found

c:\Users\riple\anaconda3\Lib\site-packages\scipy\__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 2.2.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

#### Dataset Extraction

In [ ]:
with open("Conservative.json", "r") as file:
    con_data = json.load(file)

with open("Liberal.json", "r") as file:
    lib_data = json.load(file)

con_articles = [doc["articles"] for doc in con_data]
lib_articles = [doc["articles"] for doc in lib_data]

# Preview the dataset
print(con_articles[:5])
print(lib_articles[:5])

#### Dataset Sampling

In [ ]:
# Fixed sample size of 10000
sample_size = 10000

con_sample = random.sample(con_articles, sample_size)
lib_sample = random.sample(lib_articles, sample_size)

# Preview the sampled datasets
print(con_sample[:5])
print(lib_sample[:5])

#### Text Data Preprocessing

In [ ]:
# Convert to dataframe
con_sample = pd.DataFrame({"text": con_sample})
lib_sample = pd.DataFrame({"text": lib_sample})

# Remove punctuation
con_sample["text_processed"] = con_sample["text"].map(lambda x: re.sub("[,\.!?]", "", x))
lib_sample["text_processed"] = lib_sample["text"].map(lambda x: re.sub("[,\.!?]", "", x))

# Convert the text to lowercase
con_sample["text_processed"] = con_sample["text_processed"].map(lambda x: x.lower())
lib_sample["text_processed"] = lib_sample["text_processed"].map(lambda x: x.lower())

# Preview the processed dataset
print(con_sample["text_processed"].head())
print(lib_sample["text_processed"].head())

#### POS Tagging and Tokenization

In [ ]:
hyperbolic_terms = {
    "absolutely", "completely", "totally", "utterly", "extremely", "insanely", "incredibly",
    "ridiculously", "unbelievably", "outrageously", "massively", "intensely", "tremendously",
    "overwhelmingly", "beyond", "literally",

    "amazing", "disgusting", "horrifying", "devastating", "glorious", "catastrophic", "unbelievable",
    "outrageous", "ridiculous", "perfect", "pathetic", "flawless", "disastrous", "absurd",
    "horrible", "insane", "genius", "monstrous",

    "best", "worst", "greatest", "biggest", "most", "least", "highest", "lowest", "ultimate",
    "never", "always", "forever", "all", "none", "every", "no one", "nothing", "everyone", "everything"
}

def extract_hyperbolic_terms(texts):
    results = []
    for text in texts:
        tokens = word_tokenize(text.lower()) # tokenization
        tagged = pos_tag(tokens) 
        filtered = [word for word, tag in tagged if tag in ('JJ', 'JJR', 'JJS', 'RB', 'RBR', 'RBS') and word in hyperbolic_terms]
        results.append(filtered)
    return results

con_sample['hyperbolic_terms'] = extract_hyperbolic_terms(con_sample['text'])
lib_sample['hyperbolic_terms'] = extract_hyperbolic_terms(lib_sample['text'])



#### Preparing Results

In [ ]:
# frequencies
con_freq = Counter([word for terms in con_sample['hyperbolic_terms'] for word in terms])
lib_freq = Counter([word for terms in lib_sample['hyperbolic_terms'] for word in terms])

total_con_hyperbolic = sum(con_freq.values())
total_lib_hyperbolic = sum(lib_freq.values())

print("Conservative hyperbolic language average per article:", total_con_hyperbolic/10000)
print("Liberal hyperbolic language average per article:", total_lib_hyperbolic/10000)

#### Visualized Results

In [ ]:
# Create frequency comparison table
freq_df = pd.DataFrame({
    'conservative': dict(con_freq),
    'liberal': dict(lib_freq)
}).fillna(0).astype(int)

# Normalize the frequencies by dividing by 10,000
freq_df = freq_df / 10000

# Get the top 10 most frequent words from both groups
top_words = freq_df.sum(axis=1).sort_values(ascending=False).head(10).index

# Filter the DataFrame to only include the top words
top_freq_df = freq_df.loc[top_words]

# Plot both conservative and liberal frequencies on the same bar chart
top_freq_df.plot(kind='bar', figsize=(12, 6), color=['red', 'blue'])
plt.title("Average Frequency Per Article of Top Hyperbolic Words in Conservative and Liberal Posts")
plt.ylabel("Frequency")
plt.xlabel("Words")
plt.xticks(rotation=45)
plt.legend(title="Group")
plt.tight_layout()
plt.show()